In [1]:
import gurobipy as gp
%load_ext autoreload
%autoreload 2
%aimport ModelGenerator
%aimport Methods

Set parameter Username
Set parameter LicenseID to value 2625858
Academic license - for non-commercial use only - expires 2026-02-20
Set parameter Username
Set parameter LicenseID to value 2625858
Academic license - for non-commercial use only - expires 2026-02-20
Set parameter Username
Set parameter LicenseID to value 2625858
Academic license - for non-commercial use only - expires 2026-02-20


In [2]:
mg_id = 0

In [34]:
#### Build Models
from ModelGenerator import build_models
master, subproblems, TMatrices, rVectors, Pr, Loads = build_models(mg_id=0)

Set parameter Username
Set parameter LicenseID to value 2625858
Academic license - for non-commercial use only - expires 2026-02-20


  0%|          | 0/27 [00:00<?, ?it/s]

Yeah
0.0
0.0


  4%|▎         | 1/27 [00:07<03:27,  7.98s/it]


KeyboardInterrupt: 

In [ ]:
#### Solve by Callback
from Methods import solve_with_BD_BandB
s = 2
optimal_x, optimal_obj, x_keys, num_locations = solve_with_BD_BandB(models=models[:2],
                                                                    probs=probs[:2],
                                                                    TMatrices=TMatrices[:2],
                                                                    rVectors=rVectors[:2])
optimal_x_reshaped = optimal_x.reshape(num_locations, 4)

In [ ]:
import numpy as np
x_test = np.array([20, 30, 0, 0, 0, 0, 0, 0, 0, 10, 30, 20, 0, 0, 0, 1, 0, 0, 1, 0])

In [ ]:
### Reporting
from Methods import solve_subproblems
x_u_I, x_u_E = solve_subproblems(sub_models=models,
                                 optimal_x=x_test,
                                 x_keys=x_keys,
                                 probs=probs,
                                 num_locations=num_locations,
                                 mg_id=mg_id)